#***IRIS FLOWER FEATURE EXTRACTION MODEL***

##***DataSet Collection***

https://www.kaggle.com/datasets/irijabbutt/iris-flower-plot-image-dataset

##***LOAD DATA***

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

# Load the dataset
train_dir = 'D:/Rijab/.vscode/Repos/Machine-Learning-Projects/dataset/iris_png/train'
test_dir = 'D:/Rijab/.vscode/Repos/Machine-Learning-Projects/dataset/iris_png/test'
train_images = []
train_labels = []
test_images = []
test_labels = []

for class_dir in os.listdir(train_dir):
    class_dir_path = os.path.join(train_dir, class_dir)
    for filename in os.listdir(class_dir_path):
        filepath = os.path.join(class_dir_path, filename)
        img = plt.imread(filepath)
        img = img / 255.0
        label = class_dir
    train_images.append(img)
    train_labels.append(label)

for class_dir in os.listdir(test_dir):
    class_dir_path = os.path.join(test_dir, class_dir)
    for filename in os.listdir(class_dir_path):
        filepath = os.path.join(class_dir_path, filename)
        img = plt.imread(filepath)
        img = img / 255.0
        label = class_dir
    test_images.append(img)
    test_labels.append(label)



In [2]:
train_img=np.array(train_images)
test_img=np.array(test_images)
train_lab=np.array(train_labels)
test_lab=np.array(test_labels)
print('Train Images:',train_img.shape)
print('Test Images:',test_img.shape)
print('Train Labels:',train_lab.shape)
print('Test Labels:',test_lab.shape)

Train Images: (3, 480, 640, 4)
Test Images: (3, 480, 640, 4)
Train Labels: (3,)
Test Labels: (3,)


##***DATA PREPROCESSING***

In [4]:
x_train, x_test, y_train, y_test = train_img, test_img, train_lab, test_lab
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Convert to grayscale
x_train_gray = np.dot(x_train[...,:3], [0.299, 0.587, 0.114])
x_test_gray = np.dot(x_test[...,:3], [0.299, 0.587, 0.114])

# Reshape to 2D arrays
x_train_gray = x_train_gray.reshape(x_train_gray.shape[0], -1)
x_test_gray = x_test_gray.reshape(x_test_gray.shape[0], -1)

from sklearn.preprocessing import LabelEncoder

# Convert string labels to numerical labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

##***CNN MODEL ARCHITECTURE***

In [5]:
import numpy as np
from scipy.signal import convolve2d
class Conv2D:
    
    def __init__(self, kernel_size, filters, activation, channels, input_shape):
        self.kernel_size = kernel_size
        self.filters = filters
        self.activation = activation
        self.channels = channels
        self.input_shape = input_shape
        self.weights = np.random.rand(self.kernel_size[0], self.kernel_size[1], 4, 3)
        self.bias = np.zeros((filters,))

    def forward(self, input):
        if len(input.shape) == 3:
            input = input.reshape(input.shape[0], input.shape[1], input.shape[2], 1)
        output = np.zeros((input.shape[0], input.shape[1] - self.kernel_size[0] + 1, input.shape[2] - self.kernel_size[1] + 1, self.filters))

        for j in range(input.shape[1] - self.kernel_size[0] + 1):
            for k in range(input.shape[2] - self.kernel_size[1] + 1):
                for i in range(self.filters):
                    for m in range(self.kernel_size[0]):
                        for n in range(self.kernel_size[1]):
                            if 0 <= j + m < input.shape[1] and 0 <= k + n < input.shape[2]:
                                k_n = min(k + n, input.shape[2] - 1)
                                j_m = min(j + m, input.shape[1] - 1)
                                k_idx = np.clip(k, 0, output.shape[2] - 1)
                                output[:, j_m, k_idx, i] += np.dot(input[:, j_m, k_n, 0].reshape(-1, 1), self.weights[m, n, i, :].reshape(1, -1))[0, 0]
                    output[:, j, k_idx, i] += self.bias[i]
        return output

    def backward(self, input, error):
        self.weights_grad = np.zeros_like(self.weights)
        self.bias_grad = np.zeros_like(self.bias)
        for i in range(self.filters):
            for j in range(error.shape[0]):
                for k in range(error.shape[1]):
                    self.weights_grad[:, :, :, i] += np.sum(input[j:j+self.kernel_size, k:k+self.kernel_size, :] * error[j, k, i], axis=2)
                    self.bias_grad[i] += error[j, k, i]
        return error

class MaxPool2D:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, input):
        output_shape = ((input.shape[0] + self.pool_size - 1) // self.pool_size, 
                        (input.shape[1] + self.pool_size - 1) // self.pool_size, 
                        input.shape[2])
        output = np.zeros(output_shape)
        for i in range(input.shape[2]):
            for j in range(0, input.shape[0], self.pool_size):
                for k in range(0, input.shape[1], self.pool_size):
                    output[j // self.pool_size, k // self.pool_size, i] = np.max(input[j:j+self.pool_size, k:k+self.pool_size, i])
        return output

class Flatten:
    def __init__(self):
        pass

    def forward(self, input):
        return input.reshape(input.shape[0], -1)

    def backward(self, error):
        return error.reshape(error.shape[0], -1)

class Dense:
    def __init__(self, num_inputs, units):
        self.units = units
        self.weights = np.random.rand(num_inputs, units)
        self.bias = np.zeros((units,))

    def forward(self, input):
        return np.dot(input, self.weights) + self.bias

class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, input):
        output = input
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def compile(self, optimizer, loss, metrics):
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics

    def fit(self, x_train, y_train, batch_size, epochs, validation_data, callbacks):
        history = {"epochs": [],"loss": [], "accuracy": []}
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i+batch_size]
                y_batch = y_train[i:i+batch_size]
                output = self.forward(x_batch)
                error = self.loss(y_batch, output)
                for layer in reversed(self.layers):
                    error = layer.backward(error)
                self.optimizer.update(self.layers)
            history["epochs"].append(epoch)
            history["loss"].append(error)
            history["accuracy"].append(self.metrics(y_train, self.forward(x_train)))
            
        return history

In [6]:
from sklearn.model_selection import train_test_split
batch_size = 100
epochs = 20
x_gray = np.concatenate((train_img, test_img), axis=0)
y = np.concatenate((train_lab, test_lab), axis=0)

x_train_gray, x_test_gray, y_train, y_test = train_test_split(
    x_gray, y, test_size=0.2, random_state=42
)

validation_data = (x_test_gray, y_test)
callbacks = [] 

In [7]:
# Create a neural network
nn = NeuralNetwork()
input = np.random.rand(32, 32, 3)
ch=3
# Define the model architecture
layers = []
layers.append(Conv2D(kernel_size=(3, 3), filters=8, activation='relu', channels=ch, input_shape=input))
layers.append(MaxPool2D(2))
layers.append(Conv2D(kernel_size=(3, 3), filters=8, activation='relu', channels=ch, input_shape=input))
layers.append(MaxPool2D(2))
layers.append(Conv2D(kernel_size=(3, 3), filters=8, activation='relu', channels=ch, input_shape=input))
layers.append(MaxPool2D(2))
layers.append(Flatten())
conv_output_shape = layers[-1].forward(np.zeros((1, 32, 32, 1))).shape
layers.append(Dense(num_inputs=conv_output_shape[0], units=3))

# Add the layers to the neural network
for layer in layers:
    nn.add_layer(layer)


In [8]:
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

nn.compile(optimizer=Adam(learning_rate=0.01), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])

history = nn.fit(x_train_gray, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test_gray,y_test), callbacks=callbacks)

print("Epochs:",history,"Loss:", history, "Accuracy:", history)

IndexError: index 4 is out of bounds for axis 2 with size 4

In [ ]:

# Extract features from the input image
features = nn.forward(input)

# Plot the extracted features
plt.imshow(features[0, :, :, 0], cmap='gray')
plt.show()

In [73]:
print(x_train.shape)
print(x_test.shape)
print(layer.weights.shape)

(3, 480, 640, 4)
(3, 480, 640, 4)
(1, 3)


##***CNN MODEL SUMMARY***

In [170]:
def print_model_summary(nn):
    print("Model Summary:")
    print("----------------")
    for i, layer in enumerate(nn):
        if isinstance(layer, Conv2D):
            print(f"{i+1}. Conv2D:")
            print(f"  - Filters: {layer.filters}")
            print(f"  - Kernel Size: {layer.kernel_size}")
        elif isinstance(layer, MaxPool2D):
            print(f"{i+1}. MaxPool2D:")
            print(f"  - Pool Size: {layer.pool_size}")
        elif isinstance(layer, Flatten):
            print(f"{i+1}. Flatten:")
        elif isinstance(layer, Dense):
            print(f"{i+1}. Dense:")
            print(f"  - Units: {layer.units}")
        else:
            print(f"Unknown layer type: {type(layer)}")

print_model_summary(layers)

Model Summary:
----------------
1. Conv2D:
  - Filters: 8
  - Kernel Size: (3, 3)
2. MaxPool2D:
  - Pool Size: 2
3. Conv2D:
  - Filters: 8
  - Kernel Size: (3, 3)
4. MaxPool2D:
  - Pool Size: 2
5. Conv2D:
  - Filters: 8
  - Kernel Size: (3, 3)
6. MaxPool2D:
  - Pool Size: 2
7. Flatten:
8. Dense:
  - Units: 3


##***TRAIN & TEST MODEL***

In [ ]:
# Define the batch size
batch_size = 100

# Train the model
for epoch in range(10):
    for i in range(0, len(train_images), batch_size):
        batch_inputs = train_images[i:i+batch_size]
        batch_labels = train_labels[i:i+batch_size]
        batch_outputs = []
        for input in batch_inputs:
            output = input
            for layer in layers:
                output = layer.forward(output)
            batch_outputs.append(output)
        batch_loss = np.sum((np.array(batch_outputs) - np.array(batch_labels)) ** 2)
        print(f'Epoch {epoch+1}, Batch {i//batch_size+1}, Loss: {batch_loss:.4f}')

# Test the model
test_outputs = []
for i in range(0, len(test_images), batch_size):
    batch_inputs = test_images[i:i+batch_size]
    batch_outputs = []
    for input in batch_inputs:
        output = input
        for layer in layers:
            output = layer.forward(output)
        batch_outputs.append(output)
    test_outputs.extend(batch_outputs)

##***PLOTTING RESULTS***

In [ ]:
# Plot the training and validation accuracy and loss
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

##***PERFORMANCE RESULTS***

In [ ]:
# Calculate metrics
accuracy = 0
f1 = 0
precision = 0
dice_similarity = 0
for i in range(len(test_outputs)):
     output = test_outputs[i]
     label = test_labels[i]
     accuracy += np.argmax(output) == label
     f1 += 2 * np.sum(output * label) / (np.sum(output) + np.sum(label))
     precision += np.sum(output* label) / np.sum(output)
     dice_similarity += 2 * np.sum(output * label) / (np.sum(output ** 2) + np.sum(label ** 2))
accuracy /= len(test_outputs)
f1 /= len(test_outputs)
precision /= len(test_outputs)
dice_similarity /= len(test_outputs)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Dice Similarity: {dice_similarity:.4f}')

# Plot graphs for results
plt.plot([i for i in range(len(test_outputs))], [np.argmax(output) for output in test_outputs], label='Predicted')
plt.plot([i for i in range(len(test_outputs))], test_labels, label='Ground Truth')
plt.legend()
plt.show()
